# SageMaker Principal Component Analysis (PCA) Exercise

This notebook demonstrates Amazon SageMaker's **Principal Component Analysis (PCA)** algorithm for dimensionality reduction.

## What You'll Learn
1. How to prepare data for PCA
2. How to train a PCA model
3. How to interpret and use principal components

## What is PCA?

PCA is an **unsupervised** algorithm that:
- Reduces high-dimensional data to fewer dimensions
- Finds directions of maximum variance (principal components)
- Components are orthogonal and ordered by explained variance

**SageMaker's Implementation:**
- Two modes: `regular` and `randomized`
- GPU acceleration
- Single-pass streaming
- Scalable to large datasets

## Use Cases

| Application | Description |
|-------------|-------------|
| Feature reduction | Reduce features before ML model |
| Visualization | Project high-dim data to 2D/3D |
| Noise reduction | Keep only high-variance components |
| Anomaly detection | Outliers in reduced space |
| Data compression | Store/transmit fewer features |

---

## Step 1: Setup and Imports

In [ ]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.image_uris import retrieve
from sagemaker.estimator import Estimator
import pandas as pd
import numpy as np
import json
import os
from datetime import datetime
from dotenv import load_dotenv
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification

# Load environment variables from .env file
load_dotenv()

# Configure AWS session from environment variables
aws_profile = os.getenv('AWS_PROFILE')
aws_region = os.getenv('AWS_REGION', 'us-west-2')
sagemaker_role = os.getenv('SAGEMAKER_ROLE_ARN')

if aws_profile:
    boto3.setup_default_session(profile_name=aws_profile, region_name=aws_region)
else:
    boto3.setup_default_session(region_name=aws_region)

# SageMaker session and role
sagemaker_session = sagemaker.Session()

if sagemaker_role:
    role = sagemaker_role
else:
    role = get_execution_role()

region = sagemaker_session.boto_region_name

print(f"AWS Profile: {aws_profile or 'default'}")
print(f"SageMaker Role: {role}")
print(f"Region: {region}")
print(f"SageMaker SDK Version: {sagemaker.__version__}")

In [ ]:
# Configuration
BUCKET_NAME = sagemaker_session.default_bucket()
PREFIX = "pca"

# Dataset parameters
NUM_SAMPLES = 5000
NUM_FEATURES = 50
NUM_COMPONENTS = 10
RANDOM_STATE = 42

print(f"S3 Bucket: {BUCKET_NAME}")
print(f"S3 Prefix: {PREFIX}")

## Step 2: Generate Synthetic High-Dimensional Data

In [ ]:
# Generate data with many features but only some informative
X, y = make_classification(
    n_samples=NUM_SAMPLES,
    n_features=NUM_FEATURES,
    n_informative=10,       # Only 10 features are informative
    n_redundant=20,          # 20 features are linear combinations
    n_clusters_per_class=3,
    random_state=RANDOM_STATE
)

X = X.astype(np.float32)

print(f"Data shape: {X.shape}")
print(f"Original features: {NUM_FEATURES}")
print(f"Target components: {NUM_COMPONENTS}")
print(f"\nFeature statistics:")
print(f"  Mean range: [{X.mean(axis=0).min():.2f}, {X.mean(axis=0).max():.2f}]")
print(f"  Std range: [{X.std(axis=0).min():.2f}, {X.std(axis=0).max():.2f}]")

In [ ]:
# Visualize correlation matrix
correlation = np.corrcoef(X.T)

fig, ax = plt.subplots(figsize=(10, 8))
im = ax.imshow(correlation, cmap='coolwarm', vmin=-1, vmax=1)
ax.set_title('Feature Correlation Matrix')
ax.set_xlabel('Feature')
ax.set_ylabel('Feature')
plt.colorbar(im, ax=ax)
plt.show()

print("Many features are correlated - ideal for PCA!")

## Step 3: Prepare Data for PCA

PCA expects features only in CSV or RecordIO format.

In [ ]:
# Save as CSV
os.makedirs('data/pca', exist_ok=True)

np.savetxt('data/pca/train.csv', X, delimiter=',')

print(f"Saved: data/pca/train.csv ({os.path.getsize('data/pca/train.csv') / 1024:.1f} KB)")

In [ ]:
# Upload to S3
s3_client = boto3.client('s3')

train_s3_key = f"{PREFIX}/train/train.csv"
s3_client.upload_file('data/pca/train.csv', BUCKET_NAME, train_s3_key)

train_uri = f"s3://{BUCKET_NAME}/{PREFIX}/train"
print(f"Data uploaded to: {train_uri}")

## Step 4: Train PCA Model

### Key Hyperparameters

| Parameter | Description | Default |
|-----------|-------------|---------|
| `num_components` | Number of components to compute | Required |
| `feature_dim` | Number of input features | Required |
| `mini_batch_size` | Batch size | 500 |
| `algorithm_mode` | `regular` or `randomized` | regular |
| `subtract_mean` | Center data | True |

### Mode Selection

| Mode | Best For |
|------|----------|
| `regular` | Moderate features, moderate samples |
| `randomized` | Very large features OR very large samples |

In [ ]:
# Get PCA container image
pca_image = retrieve(
    framework='pca',
    region=region,
    version='1'
)

print(f"PCA Image URI: {pca_image}")

In [ ]:
# Create PCA estimator
pca_estimator = Estimator(
    image_uri=pca_image,
    role=role,
    instance_count=1,
    instance_type='ml.m5.large',
    output_path=f's3://{BUCKET_NAME}/{PREFIX}/output',
    sagemaker_session=sagemaker_session,
    base_job_name='pca'
)

In [ ]:
# Set hyperparameters
hyperparameters = {
    "num_components": NUM_COMPONENTS,
    "feature_dim": NUM_FEATURES,
    "mini_batch_size": 500,
    "algorithm_mode": "regular",
    "subtract_mean": "True",
}

pca_estimator.set_hyperparameters(**hyperparameters)

print("PCA hyperparameters:")
for k, v in hyperparameters.items():
    print(f"  {k}: {v}")

In [ ]:
# Start training
print("Starting PCA training job...")
print("This will take approximately 3-5 minutes.\n")

pca_estimator.fit(
    {'train': train_uri},
    wait=True,
    logs=True
)

In [ ]:
# Get training job info
job_name = pca_estimator.latest_training_job.name
print(f"Training job completed: {job_name}")
print(f"Model artifacts: {pca_estimator.model_data}")

## Step 5: Deploy and Transform Data

In [ ]:
# Deploy the model
print("Deploying PCA model...")
print("This will take approximately 5-7 minutes.\n")

pca_predictor = pca_estimator.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large',
    endpoint_name=f'pca-{datetime.now().strftime("%Y%m%d%H%M")}'
)

print(f"\nEndpoint deployed: {pca_predictor.endpoint_name}")

In [ ]:
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

# Configure predictor
pca_predictor.serializer = CSVSerializer()
pca_predictor.deserializer = JSONDeserializer()

def transform_data(data, predictor, batch_size=500):
    """
    Transform data to principal components.
    """
    projections = []
    
    for i in range(0, len(data), batch_size):
        batch = data[i:i+batch_size]
        response = predictor.predict(batch)
        
        for pred in response['predictions']:
            projections.append(pred['projections'])
    
    return np.array(projections)

In [ ]:
# Transform data
print("Transforming data to principal components...")
X_transformed = transform_data(X, pca_predictor)

print(f"\nOriginal shape: {X.shape}")
print(f"Transformed shape: {X_transformed.shape}")
print(f"Dimensionality reduction: {NUM_FEATURES} -> {NUM_COMPONENTS}")

## Step 6: Analyze Principal Components

In [ ]:
# Visualize first 2 principal components
fig, ax = plt.subplots(figsize=(10, 7))

scatter = ax.scatter(X_transformed[:, 0], X_transformed[:, 1], c=y, cmap='tab10', alpha=0.6)
ax.set_xlabel('Principal Component 1')
ax.set_ylabel('Principal Component 2')
ax.set_title('Data Projected onto First 2 Principal Components')
plt.colorbar(scatter, ax=ax, label='Class')
plt.show()

In [ ]:
# Variance in each component
component_variance = np.var(X_transformed, axis=0)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Variance per component
axes[0].bar(range(NUM_COMPONENTS), component_variance)
axes[0].set_xlabel('Principal Component')
axes[0].set_ylabel('Variance')
axes[0].set_title('Variance per Principal Component')
axes[0].set_xticks(range(NUM_COMPONENTS))

# Cumulative variance
cumulative_variance = np.cumsum(component_variance) / np.sum(component_variance)
axes[1].plot(range(NUM_COMPONENTS), cumulative_variance, 'bo-')
axes[1].axhline(y=0.9, color='r', linestyle='--', label='90% variance')
axes[1].set_xlabel('Number of Components')
axes[1].set_ylabel('Cumulative Variance Explained')
axes[1].set_title('Cumulative Variance Explained')
axes[1].set_xticks(range(NUM_COMPONENTS))
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"\nVariance explained by {NUM_COMPONENTS} components: {cumulative_variance[-1]:.2%}")

In [ ]:
# 3D visualization
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

scatter = ax.scatter(X_transformed[:, 0], X_transformed[:, 1], X_transformed[:, 2], 
                    c=y, cmap='tab10', alpha=0.6)
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
ax.set_title('First 3 Principal Components')
plt.colorbar(scatter, ax=ax, label='Class', shrink=0.5)
plt.show()

## Step 7: Clean Up Resources

In [ ]:
# Delete the endpoint
print(f"Deleting endpoint: {pca_predictor.endpoint_name}")
pca_predictor.delete_endpoint()
print("Endpoint deleted successfully!")

---

## Summary

In this exercise, you learned:

1. **Data Format**: CSV with features only

2. **Key Hyperparameters**:
   - `num_components`: Target dimensions
   - `feature_dim`: Input dimensions
   - `algorithm_mode`: regular or randomized
   - `subtract_mean`: Center data

3. **Output**: Projections onto principal components

4. **Analysis**:
   - Variance per component
   - Cumulative variance explained
   - Visualization in reduced space

### Choosing Number of Components

| Method | Approach |
|--------|----------|
| Variance threshold | Keep components for 90-95% variance |
| Elbow method | Plot variance, find bend |
| Domain knowledge | Based on downstream task needs |

### Instance Recommendations

| Task | Instance Types |
|------|----------------|
| Training | ml.m5.large, ml.c5.xlarge, ml.p2.xlarge |
| Inference | ml.m5.large, ml.c5.large |

### SageMaker PCA Advantages

- GPU acceleration
- Single-pass algorithm
- Near-linear scalability
- Streaming for incremental updates

### Next Steps

- Use PCA output as input to other ML models
- Combine with K-Means for clustering
- Apply for anomaly detection in reduced space
- Visualize high-dimensional data